<a href="https://colab.research.google.com/github/sergeysh93/fa-hpc-practice/blob/master/mpisplit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install mpi4py

In [ ]:
%%writefile mpisplit.py
from mpi4py import MPI

def ring_communicate(comm):
    rank = comm.Get_rank()
    size = comm.Get_size()
    left_neighbor = (rank - 1) % size
    right_neighbor = (rank + 1) % size

    print(size, left_neighbor, rank, right_neighbor)
    data = 999
    if rank != 0:
      comm.recv(data, source=left_neighbor)
      print(f"G1.{rank}: {data} recieved from {left_neighbor}, sent to {right_neighbor}")
    comm.send(data, dest=right_neighbor)

def master_worker_communicate(comm):
    rank = comm.Get_rank()
    size = comm.Get_size()

    if rank == 0:
        # Мастер процесс
        data_to_send = list(range(1, size))
        for worker_rank in range(1, size):
            comm.send(data_to_send, dest=worker_rank)
        print(f"G2.{rank} data sent")
    else:
        # Рабочий процесс
        data_received = comm.recv(source=0)
        print(f"G2.{rank} data recieved: {data_received}")

def main():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    # Разбиение процессов на две группы
    group1 = comm.Get_group().Incl(list(range(0, size // 2)))
    group2 = comm.Get_group().Incl(list(range(size // 2, size)))

    # Создание коммуникаторов для каждой группы
    comm_group1 = comm.Create(group1)
    comm_group2 = comm.Create(group2)

    if comm_group1 != MPI.COMM_NULL:
        print(f"G1: {comm_group1.rank} / {comm_group1.size}")
        ring_communicate(comm_group1)
        comm_group1.Free()
    if comm_group2 != MPI.COMM_NULL:
        print(f"G2: {comm_group2.rank} / {comm_group2.size}")
        master_worker_communicate(comm_group2)
        comm_group2.Free()


if __name__ == "__main__":
    main()

Overwriting mpisplit.py


In [ ]:
! mpiexec --allow-run-as-root --oversubscribe -np 8 python "/content/mpisplit.py"

G1: 0 / 4
G1: 1 / 4
4 0 1 2
G1: 3 / 4
G1: 2 / 4
4 3 0 1
4 1 2 3
4 2 3 0
/content/mpisplit.py:12: UserWarning: the 'buf' argument is deprecated
  comm.recv(data, source=left_neighbor)
G1.1: 999 recieved from 0, sent to 2
G2: 1 / 4
G2: 0 / 4
G2.0 data sent
G2: 2 / 4
G2.2 data recieved: [1, 2, 3]
G2: 3 / 4
G2.3 data recieved: [1, 2, 3]
G2.1 data recieved: [1, 2, 3]
/content/mpisplit.py:12: UserWarning: the 'buf' argument is deprecated
  comm.recv(data, source=left_neighbor)
/content/mpisplit.py:12: UserWarning: the 'buf' argument is deprecated
  comm.recv(data, source=left_neighbor)
G1.2: 999 recieved from 1, sent to 3
G1.3: 999 recieved from 2, sent to 0
